# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection._search import GridSearchCV
from sklearn.model_selection._split import StratifiedKFold
from sklearn import svm


# Lendo arquivo

In [2]:
dataset = pd.read_csv("C:/Users/mcacs/Downloads/MLOps/sintomas_covid.csv",index_col=0)
dataset

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,SYMPTOM_RUNNY_NOSE,SYMPTOM_NOSE_CONGESTION,SYMPTOM_WEAKNESS,SYMPTOM_ANOSMIA_OR_HYPOSMIA,SYMPTOM_NAUSEA,SYMPTOM_LACK_OF_APPETITE,SYMPTOM_ABDOMINAL_PAIN,SYMPTOM_CONSCIOUSNESS_DEGRADATION,death,idade
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,53
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,86
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,77
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,74
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,74
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,73


# Feature Engeneering

In [3]:
def creating_age_groups(x):
    #Recém nascido
    if 0 <= x <= 5:
        x = '0 - 5'
    #Crianças
    elif 6 <= x <= 15:
        x = '6 - 15'
    #Adolescente e jovens adultos
    elif 16 <= x <= 25:
        x = '16 - 25'
    #Adultos
    elif 26 <= x <= 40:
        x = '26 - 40'
    #Meia idade
    elif 41 <= x <= 60:
        x = '41 - 60'
    #Idosos
    elif 61 <= x <= 80:
        x = '61 - 80'
    #idosos com mais de 80 anos
    else:
        x = '>80'
    
    return x
dataset['age_group'] = dataset['idade']
dataset['age_group'] = dataset['age_group'].apply(creating_age_groups)

In [4]:
dataset = pd.get_dummies(dataset, columns = ['age_group'])
dataset = dataset.drop("idade",axis=1)
dataset

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,SYMPTOM_ABDOMINAL_PAIN,SYMPTOM_CONSCIOUSNESS_DEGRADATION,death,age_group_0 - 5,age_group_16 - 25,age_group_26 - 40,age_group_41 - 60,age_group_6 - 15,age_group_61 - 80,age_group_>80
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36435 entries, 0 to 36434
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   SYMPTOM_COUGH                      36435 non-null  int64
 1   SYMPTOM_COLD                       36435 non-null  int64
 2   SYMPTOM_AIR_INSUFFICIENCY          36435 non-null  int64
 3   SYMPTOM_FEVER                      36435 non-null  int64
 4   SYMPTOM_LOW_OXYGEN_SATURATION      36435 non-null  int64
 5   SYMPTOM_BREATHING_CONDITION        36435 non-null  int64
 6   SYMPTOM_TORACIC_APERTURE           36435 non-null  int64
 7   SYMPTOM_THROAT_CONDITION           36435 non-null  int64
 8   SYMPTOM_HEADACHE                   36435 non-null  int64
 9   SYMPTOM_BODY_PAIN                  36435 non-null  int64
 10  SYMPTOM_DIARRHEA                   36435 non-null  int64
 11  SYMPTOM_RUNNY_NOSE                 36435 non-null  int64
 12  SYMPTOM_NOSE_CONGE

# Feature Selection

In [6]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, SelectFdr

In [7]:
X = dataset.drop("death",axis=1)
y = dataset['death']

# Modelos de Classificacao

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Random Forest

In [9]:
rf = RandomForestClassifier(random_state=42,n_jobs=-1)
scores = ['f1','precision','recall']
rf_results_cv = cross_validate(rf, X_train, y_train, cv=10,scoring=scores)

In [10]:
for metric in rf_results_cv:
    print(f'Média de {metric}: {rf_results_cv[metric].mean()}')

Média de fit_time: 0.74923255443573
Média de score_time: 0.15494825839996337
Média de test_f1: 0.34910341306868947
Média de test_precision: 0.5102858691179503
Média de test_recall: 0.2668854202993508


In [13]:
#pip install optuna

     -------------------------------------- 308.2/308.2 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 81.0/81.0 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 49.5/49.5 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 112.3/112.3 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 147.1/147.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 95.2/95.2 kB 1.8 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=c61b5f9e835b652fc037958664ec39745629895f9cdb8a1ba05715f05a695c65
  Stored in directory: c:\users\mcacs\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built pyperclip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python.exe -m pip in

In [11]:
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score

In [12]:
def objective(trial):

    classifier_name = trial.suggest_categorical("classifier", ["RandomForest"])

    bootstrap = trial.suggest_categorical("bootstrap", [True, False])
    #max_depth = trial.suggest_int("max_depth", 10, 100)
    criterion = trial.suggest_categorical("criterion", ['gini','entropy'])
    max_features = trial.suggest_categorical("max_features", ['auto', 'sqrt','log2'])
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    n_estimators = trial.suggest_int("n_estimators", 100, 2000)

    classifier_obj = RandomForestClassifier(
        bootstrap=bootstrap, max_depth=None,
        criterion=criterion, max_features=max_features,
        min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
        n_estimators=n_estimators, random_state=42, n_jobs=-1
    )

    score = cross_val_score(classifier_obj, X_train, y_train, n_jobs=-1, cv=5,scoring='f1')
    f1 = score.mean()
    return f1


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-08-03 22:20:23,075] A new study created in memory with name: no-name-c0d847cb-5929-4f28-98fa-e1196c55edd0
[I 2022-08-03 22:20:43,895] Trial 0 finished with value: 0.3094406782780536 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 1889}. Best is trial 0 with value: 0.3094406782780536.
[I 2022-08-03 22:20:56,987] Trial 1 finished with value: 0.2835652148269844 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 1443}. Best is trial 0 with value: 0.3094406782780536.
[I 2022-08-03 22:21:11,645] Trial 2 finished with value: 0.28414364182716334 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 3, 'n_estimators': 1623}. Best is tri

[I 2022-08-03 22:25:16,400] Trial 26 finished with value: 0.3232367125541148 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 7, 'n_estimators': 344}. Best is trial 11 with value: 0.3446286095803007.
[I 2022-08-03 22:25:37,362] Trial 27 finished with value: 0.34095977230996943 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 1361}. Best is trial 11 with value: 0.3446286095803007.
[I 2022-08-03 22:25:48,958] Trial 28 finished with value: 0.3309439626615226 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 1022}. Best is trial 11 with value: 0.3446286095803007.
[I 2022-08-03 22:26:02,534] Trial 29 finished with value: 0.3408147471920011 a

FrozenTrial(number=45, values=[0.34653805902151713], datetime_start=datetime.datetime(2022, 8, 3, 22, 27, 50, 651381), datetime_complete=datetime.datetime(2022, 8, 3, 22, 27, 57, 140615), params={'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 496}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=45, state=TrialState.COMPLETE, value=None)


In [13]:
best_params={'bootstrap': True, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 496}

best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)
for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

Média de fit_time: 5.125789833068848
Média de score_time: 0.2270437002182007
Média de test_f1: 0.34326247614818944
Média de test_precision: 0.5140697875560111
Média de test_recall: 0.2589257470559847


In [14]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)

print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

Precisão: 0.513126491646778
Revocação: 0.2377003869541183
F1-score: 0.3248961088024178


## Naive Bayes

In [15]:
from sklearn.naive_bayes import ComplementNB

In [16]:
cnb = ComplementNB()
scores = ['f1','precision','recall']
nb_results_cv = cross_validate(cnb, X_train, y_train, cv=10,scoring=scores)

In [17]:
for metric in nb_results_cv:
    print(f'Média de {metric}: {nb_results_cv[metric].mean()}')

Média de fit_time: 0.013399434089660645
Média de score_time: 0.08498849868774414
Média de test_f1: 0.5440745899203498
Média de test_precision: 0.432252582692254
Média de test_recall: 0.7342997594062302


In [18]:
def objective(trial):

    classifier_name = trial.suggest_categorical("classifier", ["ComplementNB"])

    fit_prior = trial.suggest_categorical("fit_prior", [True, False])
    alpha = trial.suggest_float("alpha", 0.001, 1)
    norm = trial.suggest_categorical("norm", [True, False])

    classifier_obj = ComplementNB(
        fit_prior=fit_prior, alpha=alpha, norm=norm
    )

    score = cross_val_score(classifier_obj, X_train, y_train, n_jobs=-1, cv=5,scoring='f1')
    f1 = score.mean()
    return f1


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-08-03 22:31:42,366] A new study created in memory with name: no-name-e6474593-9c7d-406d-9c4f-b7f66e57db28
[I 2022-08-03 22:31:42,550] Trial 0 finished with value: 0.5373904779041514 and parameters: {'classifier': 'ComplementNB', 'fit_prior': False, 'alpha': 0.6721118852192909, 'norm': True}. Best is trial 0 with value: 0.5373904779041514.
[I 2022-08-03 22:31:42,753] Trial 1 finished with value: 0.5430433931005488 and parameters: {'classifier': 'ComplementNB', 'fit_prior': True, 'alpha': 0.5635121182635996, 'norm': False}. Best is trial 1 with value: 0.5430433931005488.
[I 2022-08-03 22:31:42,924] Trial 2 finished with value: 0.5430433931005488 and parameters: {'classifier': 'ComplementNB', 'fit_prior': True, 'alpha': 0.6373520860713866, 'norm': False}. Best is trial 1 with value: 0.5430433931005488.
[I 2022-08-03 22:31:43,097] Trial 3 finished with value: 0.5429001182788059 and parameters: {'classifier': 'ComplementNB', 'fit_prior': False, 'alpha': 0.50877765810132, 'norm': Fal

[I 2022-08-03 22:31:48,531] Trial 33 finished with value: 0.5430433931005488 and parameters: {'classifier': 'ComplementNB', 'fit_prior': True, 'alpha': 0.6297803095768688, 'norm': False}. Best is trial 1 with value: 0.5430433931005488.
[I 2022-08-03 22:31:48,703] Trial 34 finished with value: 0.5430433931005488 and parameters: {'classifier': 'ComplementNB', 'fit_prior': False, 'alpha': 0.5197797498165078, 'norm': False}. Best is trial 1 with value: 0.5430433931005488.
[I 2022-08-03 22:31:48,875] Trial 35 finished with value: 0.5373502033228303 and parameters: {'classifier': 'ComplementNB', 'fit_prior': False, 'alpha': 0.5443463270329306, 'norm': True}. Best is trial 1 with value: 0.5430433931005488.
[I 2022-08-03 22:31:49,060] Trial 36 finished with value: 0.5430433931005488 and parameters: {'classifier': 'ComplementNB', 'fit_prior': False, 'alpha': 0.536908286900427, 'norm': False}. Best is trial 1 with value: 0.5430433931005488.
[I 2022-08-03 22:31:49,243] Trial 37 finished with valu

FrozenTrial(number=1, values=[0.5430433931005488], datetime_start=datetime.datetime(2022, 8, 3, 22, 31, 42, 553350), datetime_complete=datetime.datetime(2022, 8, 3, 22, 31, 42, 752748), params={'classifier': 'ComplementNB', 'fit_prior': True, 'alpha': 0.5635121182635996, 'norm': False}, distributions={'classifier': CategoricalDistribution(choices=('ComplementNB',)), 'fit_prior': CategoricalDistribution(choices=(True, False)), 'alpha': UniformDistribution(high=1.0, low=0.001), 'norm': CategoricalDistribution(choices=(True, False))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)


In [19]:
best_params={'fit_prior': True, 'alpha': 0.5635121182635996, 'norm': False}

best_nb = ComplementNB(**best_params)
scores = ['f1','precision','recall']
best_nb_results_cv = cross_validate(best_nb, X_train, y_train, cv=10,scoring=scores)
for metric in best_nb_results_cv:
    print(f'Média de {metric}: {best_nb_results_cv[metric].mean()}')

Média de fit_time: 0.012622809410095215
Média de score_time: 0.006293535232543945
Média de test_f1: 0.5441525522258035
Média de test_precision: 0.4323495610181176
Média de test_recall: 0.7342997594062302


In [20]:
best_nb.fit(X_train, y_train)
y_pred = best_nb.predict(X_test)

print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

Precisão: 0.4288939051918736
Revocação: 0.7352128247650636
F1-score: 0.5417515274949083


# MLFlow

In [21]:
import mlflow

In [22]:
import pickle

## Experiment Tracking

In [23]:
mlflow.set_tracking_uri("sqlite:///mlops.db")
mlflow.set_experiment("projeto_final")

2022/08/03 22:33:01 INFO mlflow.tracking.fluent: Experiment with name 'projeto_final' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='projeto_final', tags={}>

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Random Forest

In [48]:
with open('C:/Users/mcacs/Downloads/MLOps/random_forest.bin', 'wb') as f_out:
    pickle.dump(rf, f_out)

In [49]:
mlflow.sklearn.autolog()
with mlflow.start_run():

    mlflow.set_tag("model", "Random Forest")
    
    mlflow.log_param("train-data-path", pd.concat([X_train,y_train],axis=1))
    mlflow.log_param("valid-data-path", pd.concat([X_valid,y_valid],axis=1))

    best_params={'bootstrap': True, 
                 'criterion': 'entropy', 
                 'max_features': 'log2', 
                 'min_samples_leaf': 1, 
                 'min_samples_split': 2, 
                 'n_estimators': 496}
    
    mlflow.log_param("parameters", best_params)
    
    rf = RandomForestClassifier(**best_params)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_valid)
    f1 = f1_score(y_valid,y_pred)
    
    mlflow.log_metric("f1", f1)

    mlflow.log_artifact(local_path="C:/Users/mcacs/Downloads/MLOps/random_forest.bin", artifact_path="models_pickle")

2022/08/03 23:00:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\mcacs\anaconda3\lib\site-packages\mlflow\models\signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


### Naive Bayes

In [50]:
with open('C:/Users/mcacs/Downloads/MLOps/naive_bayes.bin', 'wb') as f_out:
    pickle.dump(cnb, f_out)

In [51]:
mlflow.sklearn.autolog()
with mlflow.start_run():

    mlflow.set_tag("model", "Naive Bayes")
    
    mlflow.log_param("train-data-path", pd.concat([X_train,y_train],axis=1))
    mlflow.log_param("valid-data-path", pd.concat([X_valid,y_valid],axis=1))

    best_params={'fit_prior': True, 'alpha': 0.5635121182635996, 
                 'norm': False}
    
    mlflow.log_param("alpha", best_params)
    
    cnb = ComplementNB(**best_params)
    cnb.fit(X_train, y_train)

    y_pred = cnb.predict(X_valid)
    f1 = f1_score(y_valid,y_pred)
    
    mlflow.log_metric("f1", f1)

    mlflow.log_artifact(local_path="C:/Users/mcacs/Downloads/MLOps/naive_bayes.bin", artifact_path="models_pickle")

2022/08/03 23:02:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\mcacs\anaconda3\lib\site-packages\mlflow\models\signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2022/08/03 23:02:11 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred whil

## Model Registry

### Listando os modelos salvos

In [52]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlops.db"

In [53]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='projeto_final', tags={}>]

In [54]:
client.get_experiment(experiment_id="1")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='projeto_final', tags={}>

In [55]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.f1 > 0.1",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.f1 ASC"]
)

In [56]:
for run in runs:
    print(f"run id: {run.info.run_id}, f1: {run.data.metrics['f1']:.4f}")

run id: 4e0b333c69d14bb990a544fdf3630bd2, f1: 0.3416
run id: 18aa0aa3b3fe4a1998dd90c635ae1206, f1: 0.3468
run id: fbe822ea478a474f879a781234a061dd, f1: 0.5429
run id: fe7335fcd81141efa2b861a8edec48c7, f1: 0.5429


### Definindo o estagio dos modelos

In [57]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [61]:
run_id_nb = "fbe822ea478a474f879a781234a061dd"
model_uri = f"runs:/{run_id_nb}/model"
mlflow.register_model(model_uri=model_uri, name="projeto_final_classifier")

Registered model 'projeto_final_classifier' already exists. Creating a new version of this model...
2022/08/03 23:10:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: projeto_final_classifier, version 7
Created version '7' of model 'projeto_final_classifier'.


<ModelVersion: creation_timestamp=1659579059152, current_stage='None', description=None, last_updated_timestamp=1659579059152, name='projeto_final_classifier', run_id='fbe822ea478a474f879a781234a061dd', run_link=None, source='./mlruns/1/fbe822ea478a474f879a781234a061dd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [62]:
model_version = 7
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1659579059152, current_stage='Staging', description=None, last_updated_timestamp=1659579068078, name='projeto_final_classifier', run_id='fbe822ea478a474f879a781234a061dd', run_link=None, source='./mlruns/1/fbe822ea478a474f879a781234a061dd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [63]:
run_id_rf = "4e0b333c69d14bb990a544fdf3630bd2"
model_uri = f"runs:/{run_id_rf}/model"
mlflow.register_model(model_uri=model_uri, name="projeto_final_classifier")

Registered model 'projeto_final_classifier' already exists. Creating a new version of this model...
2022/08/03 23:11:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: projeto_final_classifier, version 8
Created version '8' of model 'projeto_final_classifier'.


<ModelVersion: creation_timestamp=1659579095961, current_stage='None', description=None, last_updated_timestamp=1659579095961, name='projeto_final_classifier', run_id='4e0b333c69d14bb990a544fdf3630bd2', run_link=None, source='./mlruns/1/4e0b333c69d14bb990a544fdf3630bd2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=8>

In [64]:
model_name = "projeto_final_classifier"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 8, stage: None
version: 7, stage: Staging


In [67]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"f1": f1_score(y_test, y_pred)}

In [68]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

Wall time: 48.7 ms


{'f1': 0.539766918830505}

In [69]:
%time test_model(name=model_name, stage="None", X_test=X_test, y_test=y_test)

Wall time: 746 ms


{'f1': 0.33458646616541354}

In [70]:
client.transition_model_version_stage(
    name=model_name,
    version=7,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1659579059152, current_stage='Production', description=None, last_updated_timestamp=1659579191071, name='projeto_final_classifier', run_id='fbe822ea478a474f879a781234a061dd', run_link=None, source='./mlruns/1/fbe822ea478a474f879a781234a061dd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [71]:
model_name = "projeto_final_classifier"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 8, stage: None
version: 4, stage: Staging
version: 7, stage: Production
